# Code Transcribe 
* Langchain
* LangSmith
* Semantic Kernel
* Async Calls
* Build APIs
* Integrate Airflow, DB, Logging

Models:
* Mistral Dolphin: 
* 

---

## 1. Install and Load Dependencies

---

## 2. Load PIG Code (optionsal: Upload sample data)

---

## 3. 

---

In [1]:
# https://deepgram.com/learn/local-llm-chatbot-that-can-run-code-searches

In [2]:
# A.1. LLM transcriber PIG2PySpark
# Prompt: 
# You are an experienced Software Engineer and Machine Learning Engineer fluent in PIG and PySpark coding languages. 
# Rewrite the following PIG code into PySpark so that they can perform identical tasks and output identical results given same input data. 
# PIG Code: {pig_code}
# A.2. Code Parser: Parse and save PySpark code 

# B. Sample Data Builder 
# C. PIG Interpreter 
# D. PySpark Interpreter 

# If sample data NOT available (run B.)
  # Run PIG code and generate output 
  # 1. Generate PySpark code from PIG (run A.1) and parse/save the PySpark Code (run A.2)
# 2. Run against sample data in a separate module.
# 3. Check 

In [3]:
import os
# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current working directory:", current_directory)


Current working directory: /workspace


In [4]:
import subprocess
import os

def run_pig_script(script_path, data_path):
    # Set up the environment variable to point to the directory containing the data file
    os.environ['PIG_DATA_PATH'] = data_path

    # Execute the Pig script using subprocess, assuming Pig is installed and configured to run in local mode
    result = subprocess.run(['pig', '-x', 'local', '-f', script_path], capture_output=True, text=True)
    
    # Check the results of the Pig script execution
    if result.returncode != 0:
        print("Error occurred during Pig script execution:")
        print(result.stderr)
    else:
        print("Pig script executed successfully. Output:")
        print(result.stdout)

# Define the path to the Pig script and the directory containing the data file
pig_script = './scripts/pig1.pig'
csv_data = './data/'

# Execute the Pig script
run_pig_script(pig_script, csv_data)


Error occurred during Pig script execution:
2024-05-02 23:05:26,996 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-05-02 23:05:26,996 [main] INFO  org.apache.pig.Main - Logging error messages to: /workspace/pig_1714691126990.log
2024-05-02 23:05:27,015 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-05-02 23:05:27,261 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-05-02 23:05:27,322 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-05-02 23:05:27,323 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2024-05-02 23:05:27,341 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session: PIG-pig1.pig-49022241-9885-4fc4-ba9e

In [5]:
import pandas as pd

# Path to the output file created by Pig
output_file = 'output/high_value_transactions/part-m-00000'  # Adjust path as needed

# Read the CSV file into a DataFrame
df = pd.read_csv(output_file, names=['depStore', 'total_sales'])
df.head()


,depStore,total_sales
store1,2022-01-02,300
store2,2022-01-02,250
store3,2022-01-01,400
store3,2022-01-02,500


---

## Building LLM Layer - Pig2PySpark 
* https://medium.com/@yash9439/unleashing-the-power-of-falcon-code-a-comparative-analysis-of-implementation-approaches-803048ce65dc
* ref: https://medium.com/@ajay_khanna/leveraging-llama2-0-for-question-answering-on-your-own-data-using-cpu-aa6f75868d2d
* ref: https://medium.com/@murtuza753/using-llama-2-0-faiss-and-langchain-for-question-answering-on-your-own-data-682241488476
* https://wellsr.com/python/fine-tuning-llama2-for-question-answering-tasks/
* https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb

"-- Load the data from a CSV file\ntransactions = LOAD 'data/sample1.csv' USING PigStorage(',') \n    AS (depStore:chararray, date:chararray, amount:int);\n\n-- Filter transactions to include only those where the amount is greater than 200\nhigh_value_transactions = FILTER transactions BY amount > 200;\n\n-- Group the transactions by store\ngrouped_by_store = GROUP high_value_transactions BY depStore;\n\n-- Calculate total and average sales per depStore\nsales_summary = FOREACH grouped_by_store GENERATE \n    group AS depStore,\n    SUM(high_value_transactions.amount) AS total_sales,\n    AVG(high_value_transactions.amount) AS average_sales;\n\n-- Store the summary in a CSV file\nSTORE sales_summary INTO 'output/sales_summary' USING PigStorage(',');\n\n-- Optional: Just for demonstration, store filtered data to another directory\nSTORE high_value_transactions INTO 'output/high_value_transactions' USING PigStorage(',');\n"

In [1]:
import transformers
import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


# messages = [
#     {"role": "system", "content": "You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark."},
#     {"role": "user", "content": f"Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"},
# ]

messages = [
    {"role": "user", "content": f"You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark. Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])



/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fedf181d1d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fedf181d1d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
import langchain as lc

# Define the model ID and initialize the LLM
model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"
llm = lc.LLM.from_pretrained(
    model_id=model_id,
    device_map="auto",
    torch_dtype="bfloat16"
)

# Initial prompt setup
initial_prompt = "You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark. Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"

# Define a function to execute PySpark code and capture any errors
def execute_pyspark_code(code):
    try:
        # Assuming a function `execute_code` that runs PySpark code and returns results or errors
        result = execute_code(code)
        return result, None
    except Exception as e:
        return None, str(e)

# Define a function to refine code using error messages
def refine_code_with_error(code, error_message):
    feedback_prompt = f"{initial_prompt} However, the following error occurred when executing your previous PySpark code: {error_message} Please fix it. Previous PySpark code: {code}"
    messages = [{"role": "user", "content": feedback_prompt}]
    result = llm_chain.run(input=messages)
    return result.generated_text

# Create a chain for generating PySpark code
llm_chain = lc.Chain(
    steps=[
        lc.steps.TextGeneration(
            llm=llm,
            max_new_tokens=256,
            temperature=0.6,
            top_p=0.9,
            end_tokens=["", llm.tokenizer.eos_token]
        )
    ]
)

# Initial code generation
initial_messages = [{"role": "user", "content": initial_prompt}]
initial_result = llm_chain.run(input=initial_messages)
generated_code = initial_result.generated_text

# Attempt to execute the generated code
execution_result, error = execute_pyspark_code(generated_code)

# If an error occurs, refine the code
if error:
    refined_code = refine_code_with_error(generated_code, error)
    print("Refined PySpark Code:", refined_code)
else:
    print("Successfully executed PySpark Code:", execution_result)


In [ ]:
# import transformers
# import torch

# # model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "gradientai/Llama-3-8B-Instruct-Gradient-1048k"

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )


# # messages = [
# #     {"role": "system", "content": "You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark."},
# #     {"role": "user", "content": f"Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"},
# # ]

# messages = [
#     {"role": "user", "content": f"You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark. Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"},
# ]

# prompt = pipeline.tokenizer.apply_chat_template(
#         messages, 
#         tokenize=False, 
#         add_generation_prompt=True
# )

# terminators = [
#     pipeline.tokenizer.eos_token_id,
#     pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = pipeline(
#     prompt,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )
# print(outputs[0]["generated_text"][len(prompt):])



In [ ]:
# import transformers
# import torch

# # model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )


# # messages = [
# #     {"role": "system", "content": "You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark."},
# #     {"role": "user", "content": f"Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"},
# # ]

# messages = [
#     {"role": "user", "content": f"You are a highly experienced software engineer and machine learning engineer fluent in both PIG and PySpark. Re-write the following PIG code into PySpark code. Make sure to only share PySpark so that the two codes are logically identical and output identical results given input. PIG code: {pig_script_code}"},
# ]

# prompt = pipeline.tokenizer.apply_chat_template(
#         messages, 
#         tokenize=False, 
#         add_generation_prompt=True
# )

# terminators = [
#     pipeline.tokenizer.eos_token_id,
#     pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = pipeline(
#     prompt,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )
# print(outputs[0]["generated_text"][len(prompt):])



In [ ]:
# Sure! Below is a Python code template that uses an LLM to convert PIG code to PySpark code, then automatically tests and refines the PySpark code based on feedback. This example assumes that you are using an API to interact with an LLM and are capable of running PySpark scripts programmatically. Here, I’ll outline a simplified version that focuses on the core loop and error handling.

# ### Python Code Example

# ```python
# import requests

# def get_pyspark_code(pig_code):
#     """ Send PIG code to LLM and receive initial PySpark translation """
#     response = requests.post('https://api.example.com/translate', json={'code': pig_code})
#     return response.json()['pyspark_code']

# def test_pyspark_code(pyspark_code, test_input):s
#     """
#     Run PySpark code and return the output.
#     This function would realistically run the code in a separate environment and capture outputs.
#     """
#     # Placeholder for actual code execution and output capturing
#     # You would replace this with actual execution logic
#     output = run_pyspark_script(pyspark_code, test_input)
#     return output

# def run_pyspark_script(pyspark_code, input_data):
#     """ A mock function to execute PySpark code - replace with actual execution logic """
#     # This would involve setting up Spark session and running the code
#     # For simplicity, this is just a placeholder
#     return "Mock output based on " + input_data

# def main(pig_code, test_input, expected_output):
#     pyspark_code = get_pyspark_code(pig_code)
#     attempts = 0
#     max_attempts = 10  # Prevent infinite loops by limiting the number of attempts

#     while attempts < max_attempts:
#         attempts += 1
#         output = test_pyspark_code(pyspark_code, test_input)
#         if output == expected_output:
#             print("Success! PySpark code matches the expected output.")
#             break
#         else:
#             print("Mismatch or error detected. Refining code...")
#             # Send feedback and get refined code
#             response = requests.post('https://api.example.com/refine', json={
#                 'original_code': pyspark_code,
#                 'output': output,
#                 'expected_output': expected_output
#             })
#             pyspark_code = response.json()['refined_pyspark_code']
#     else:
#         print("Maximum attempts reached. Final PySpark code may need manual review.")

# if __name__ == "__main__":
#     pig_code = "LOAD 'data' AS (a:int, b:int, c:int); FILTER a by a > 0; STORE a INTO 'output';"
#     test_input = "Sample input data that your PIG code expects"
#     expected_output = "Expected output format or data"
#     main(pig_code, test_input, expected_output)
# ```

# ### Key Components:

# 1. **API Interaction**: This script assumes the existence of two endpoints: one to translate PIG to PySpark, and another to refine the PySpark code based on feedback.
# 2. **Execution and Testing**: It simplistically represents the PySpark execution. In a real-world scenario, this would involve using a `subprocess` to run a Spark job, capturing its output, and perhaps even interacting with a test database or data storage.
# 3. **Loop and Feedback**: The script uses a loop to refine the PySpark code based on mismatches between expected and actual outputs, up to a maximum number of iterations to prevent infinite loops.

# ### Enhancements for Production:

# - **Error Handling**: Implement robust error handling around API calls and PySpark execution.
# - **Logging**: Add detailed logging for each step to facilitate debugging and monitoring.
# - **Security**: Secure API interactions and handle data securely, especially when working with sensitive or large datasets.

In [ ]:
# ===============================================================================================================
# parse Python code inside the code cell (TODO: How to ensure code is consistently inside the triple back ticks?) 
# How to loop the code so that it runs until correct code is written? 
# How does Langchain come into play? 
# When debugging: 1) provide loaded data head 2) code 3) error message or output if run was successful --> output updated code
# ===============================================================================================================

In [ ]:
# from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
# import torch

# # Define the model
# # model_name = "tiiuae/falcon-7b-instruct"

# # Save model and tokenizer to a specific directory (optional)
# # model.save_pretrained('./model_cache/falcon-7b-instruct')
# # tokenizer.save_pretrained('./model_cache/falcon-7b-instruct')

# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# # Download and cache model and tokenizer
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model.save_pretrained('./model_cache/Meta-Llama-3-8B-Instruct')
# del model 

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.save_pretrained('./model_cache/Meta-Llama-3-8B-Instruct')
# del tokenizer

---
---
---

In [ ]:
# # Load the model and tokenizer from the cache for use
# tokenizer = AutoTokenizer.from_pretrained('./model_cache/Meta-Llama-3-8B-Instruct')
# model = AutoModelForCausalLM.from_pretrained('./model_cache/Meta-Llama-3-8B-Instruct')

# # Setup the pipeline with local model and tokenizer
# text_generation = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device=0  # Assuming using the first GPU
# )

# # Generate text
# generated_text = text_generation("Sample prompt text goes here", max_length=50)
# print(generated_text)

In [ ]:
# question = f"Re-write the following PIG code into PySpark code. Following is the PIG code: \n {pig_script_code}"
# template = f"""
# You are an intelligent software engineer and machine learning engineer. Re-write the following PIG code into PySpark code. Make sure to only share PySpark code so it's easy to copy and paste. 
# PIG code: {question}
# --------------------------------------------------------------------
# PySpark code:"""

# sequences = pipeline(
#     template,
#     max_length=5000,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
# )

# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")

---
## Test PySpark Code

In [ ]:
def build_prompt_starter(): 
    """
    Generate prompt to start transcribing PIG to PySpark.
    """

def build_prompt_fix_error(): 
    """
    Generate prompt to fix error in PySpark code.
    """

def build_prompt_fix_output(): 
    """
    Generate prompt to fix code output mismatch (between result from PIG and PySpark). 
    """
    

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg


def run_pyspark_code(code_dir, code_name, sample_data_dir): 
    # start PySpark server
    # run code against test data

    if error: 
        return error_message
    else: 
        return result_df

def check_resutls(pig_result_df, pyspark_result_df): 
    # return True if the results are identical 
    return is_same 


# loop until both pyspark code runs fine and output data from PIG and PYSpark are the same
while not is_pyspark_pass or not is_result_identical:  
    


# Create a SparkSession
spark = SparkSession.builder.appName("Sales Summary").getOrCreate()

# Load the data from a CSV file
transactions = spark.read.csv('data/sample1.csv', header=False, inferSchema=True)

# Define your list of column names
column_names = ["store", "date", "amount"]

# Filter transactions to include only those where the amount is greater than 200
high_value_transactions = transactions.filter(transactions.amount > 200)

# Group the transactions by store
grouped_by_store = high_value_transactions.groupBy("depStore")

# Calculate total and average sales per depStore
sales_summary = grouped_by_store.agg({"amount": "sum"}).alias("total_sales"), avg("amount").alias("average_sales")

# Store the summary in a CSV file
sales_summary.write.csv('output/sales_summary', header=True, sep=',')

# Optional: Just for demonstration, store filtered data to another directory
high_value_transactions.write.csv('output/high_value_transactions', header=True, sep=',')

In [ ]:
from pyspark.sql import SparkSession
from tabulate import tabulate

def build_prompt_starter(pig_code):
    """
    Generate prompt to start transcribing PIG to PySpark.
    Returns a string with the starter prompt.
    """
    return f"""
    I need to convert the following Apache PIG script into Apache PySpark code. 
    The PySpark code should perform the same tasks and produce identical outputs as the PIG code. 
    Please ensure that the PySpark code uses DataFrame operations wherever possible and include comments explaining any complex parts or transformations.
    
    PIG Code: 
    {pig_code}
    """


def build_prompt_fix_error(error_message):
    """
    Generate prompt to fix error in PySpark code.
    Returns a string with the prompt to fix errors.
    """
    return f"""There was an error in your PySpark code: {error_message}. Please fix it and share an updated PySpark code."""



pig_output = tabulate(pig_output, headers='keys', tablefmt='psql', showindex="never")
pyspark_output = tabulate(pyspark_output, headers='keys', tablefmt='psql', showindex="never")


def build_prompt_fix_output(pig_output, pyspark_output):
    """
    Generate prompt to fix code output mismatch between results from PIG and PySpark.
    Returns a string with the prompt for fixing output mismatches.
    """
    return f"""
    The outputs between PIG and PySpark do not match. Please adjust the PySpark code so that the output from PySpark code is identical to that from PIG code. 
    Below are the two outputs with mismatch:

    Pig Output (ground truth): 
    {pig_output}

    
    PySpark Output: 
    {pyspark_output}
    """


def run_pyspark_code(code):
    """
    Executes PySpark code, returns either error message or result DataFrame.
    Assumes PySpark session and context are already set.
    """
    try:
        exec(code)
        return None, globals().get('sales_summary')  # Assuming 'sales_summary' is the result DataFrame
    except Exception as e:
        return str(e), None


def check_results(pig_result_df, pyspark_result_df):
    """
    Compare PIG and PySpark DataFrames to check if the results are identical.
    """
    return pig_result_df.subtract(pyspark_result_df).count() == 0 and pyspark_result_df.subtract(pig_result_df).count() == 0


# Set up PySpark
spark = SparkSession.builder.appName("Sales Summary").getOrCreate()

# Initial PySpark code generation using an LLM (not shown here)
pyspark_code = """
# Assume pyspark_code is filled with the initially generated code
"""
pig_result_df = spark.createDataFrame(...)  # Assume this is setup elsewhere

# Run and refine PySpark code
error_message, pyspark_result_df = run_pyspark_code(pyspark_code)
while error_message or not check_results(pig_result_df, pyspark_result_df):
    if error_message:
        prompt = build_prompt_fix_error(error_message)
    else:
        prompt = build_prompt_fix_output()
    # Here you would update `pyspark_code` based on the LLM's output (not shown here)
    error_message, pyspark_result_df = run_pyspark_code(pyspark_code)

# Results are now fine
print("PySpark code executed successfully and results match PIG.")


In [ ]:
# subprocess to test out sample codes: 
# link: https://www.google.com/search?q=Python+application+which+run+PIG+code&rlz=1C1OPNX_enUS1108US1108&oq=Python+application+which+run+PIG+code+&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigAdIBCTEwNTIzajBqN6gCALACAA&sourceid=chrome&ie=UTF-8

# langchain 

# langsmith 

# streamlit 

# airflow 

# 